<a href="https://colab.research.google.com/github/raaraya1/Personal-Proyects/blob/main/Cursos/Gurobi/Facility_location_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Problema de Facility Location (Solver Gurobi)**

**Contexto:** Tengo que decidir donde abrir una bodega en funcion de minimizar los costos de tranporte y los costos de instalacion de la misma. Ademas, se pide extraer la configuracion optima de como realizar los despachos a los clientes.


**Fuente:** https://www.gurobi.com/resource/facility-location-problem/


# **Modelo**

**Conjuntos:**

$i \in I$ : Conjunto de clientes

$j \in J$ : Conjunto de posibles locaciones para la bodegas.

**Parametros:**

$Cinst_{j}$ : Costo de instalacion de la bodega j.

$Dist_{ij}$ : Distancia entre el cliente i y la bodega j.

$Cbenc$: Costo de la bencina por kilometro.

**Variables:**

$X_{j} \in (0, 1)$: Si escojo instalar la bodeja j

$Y_{ij}$: Porcentaje del pedido del cliente i que satisface la bodega j. (entre 0 a 1)

**Funcion Objetivo:**

\begin{equation}
Min \sum_{j} Cinst_{j}X_{j} + \sum_{i}\sum_{j} Cbenc Dist_{ij} Y_{ij}
\end{equation}

**Restricciones:**

1) Tengo que completar el pedido de los clientes.

\begin{equation}
\sum_{j}Y_{i} = 1 \quad \forall i \in I
\end{equation}

2) No puedo transportar mercancia a un cliente desde una planta inhabilitada.

\begin{equation}
Y_{ij} \leq X_{j} \quad \forall j \in J, i \in I
\end{equation}


In [ ]:
# Installar biblioteca de gurobi
%pip install gurobipy

     |████████████████████████████████| 11.1 MB 135 kB/s 


In [ ]:
# Importar bibliotecas necesarias
from gurobipy import Model, quicksum, GRB
import pandas as pd

In [ ]:
# Cargar los datos necesarios
!ls

Facility_location_Gurobi.xlsx  sample_data


In [ ]:
# Extraer los datos del excel 
## Me voy a saltar las primeras 8 columnas
df = pd.read_excel('/content/Facility_location_Gurobi.xlsx', usecols=[8, 9, 10])
df

,Cliente i.1,Bodega j.1,Dist_{ij}
0,1,1,126
1,1,2,123
2,1,3,209
3,1,4,90
4,1,5,295
...,...,...,...
175,20,5,211
176,20,6,269
177,20,7,301
178,20,8,108


In [ ]:
# Este df lo voy a pasar a un diccionario, para interactuar mas facilmente con los datos
dist_ij = {}
for i in range(len(df)):
  dist_ij[(df['Cliente i.1'][i], df['Bodega j.1'][i])] = float(df['Dist_{ij}'][i])
print(dist_ij)
print(type(dist_ij[(1, 1)]))

{(1, 1): 126.0, (1, 2): 123.0, (1, 3): 209.0, (1, 4): 90.0, (1, 5): 295.0, (1, 6): 261.0, (1, 7): 266.0, (1, 8): 107.0, (1, 9): 66.0, (2, 1): 162.0, (2, 2): 50.0, (2, 3): 136.0, (2, 4): 164.0, (2, 5): 233.0, (2, 6): 124.0, (2, 7): 109.0, (2, 8): 136.0, (2, 9): 196.0, (3, 1): 63.0, (3, 2): 172.0, (3, 3): 123.0, (3, 4): 93.0, (3, 5): 149.0, (3, 6): 216.0, (3, 7): 253.0, (3, 8): 91.0, (3, 9): 132.0, (4, 1): 243.0, (4, 2): 140.0, (4, 3): 181.0, (4, 4): 251.0, (4, 5): 261.0, (4, 6): 114.0, (4, 7): 69.0, (4, 8): 221.0, (4, 9): 287.0, (5, 1): 78.0, (5, 2): 161.0, (5, 3): 187.0, (5, 4): 51.0, (5, 5): 248.0, (5, 6): 267.0, (5, 7): 289.0, (5, 8): 85.0, (5, 9): 32.0, (6, 1): 101.0, (6, 2): 14.0, (6, 3): 107.0, (6, 4): 102.0, (6, 5): 206.0, (6, 6): 140.0, (6, 7): 147.0, (6, 8): 74.0, (6, 9): 139.0, (7, 1): 203.0, (7, 2): 151.0, (7, 3): 102.0, (7, 4): 227.0, (7, 5): 153.0, (7, 6): 10.0, (7, 7): 47.0, (7, 8): 192.0, (7, 9): 276.0, (8, 1): 152.0, (8, 2): 136.0, (8, 3): 43.0, (8, 4): 182.0, (8, 5): 10

In [ ]:
# Coloquemos el resto de los parametros
Cbenc = 1
Cinst_j = {1:400,
           2:250,
           3:100,
           4:300,
           5:150,
           6:200,
           7:120,
           8:600,
           9:700}

In [ ]:
# Ahora armemos el modelo

N_clientes = 20
N_bodegas = 9

m = Model()

# Variables
X = {}
for j in range(N_bodegas):
  X[j] = m.addVar(vtype=GRB.BINARY)

Y = {}
for i in range(N_clientes):
  for j in range(N_bodegas):
    Y[i, j] = m.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1)

Z = m.addVar(vtype=GRB.CONTINUOUS)

# Restricciones

## Satisfacer la demanda
for i in range(N_clientes):
  m.addConstr(quicksum(Y[i, j] for j in range(N_bodegas)) == 1)

## No transportar si no se encuentra habilitado la bodega
for i in range(N_clientes):
  for j in range(N_bodegas):
    m.addConstr(Y[i, j] <= X[j])

# Asignar objetivo
m.addConstr(Z == 
            quicksum(Cinst[j+1]*X[j] for j in range(N_bodegas)) + 
            quicksum(Cbenc*dist_ij[(i+1, j+1)]*Y[i, j] for i in range(N_clientes) for j in range(N_bodegas)))
m.setObjective(Z, GRB.MINIMIZE)
m.optimize() 


Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 201 rows, 190 columns and 730 nonzeros
Model fingerprint: 0xe47ed68a
Variable types: 181 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 200 rows, 189 columns, 540 nonzeros
Variable types: 180 continuous, 9 integer (9 binary)

Root relaxation: objective 1.993000e+03, 40 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1993.0000000 1993.00000  0.00%     -    0s

Explored 0 nodes (40 simplex iterations) in 0.05 seconds
Thread count was 2 (of 2 available processors)

S

In [ ]:
# Ahora, para mirar cuales son las variables que se utilizan para generar este resultado
## Resultados varaible X
for j in range(N_bodegas):
  if X[j].x > 0:
    print('X{}: {}'.format(j+1, X[j].x))

X2: 1.0
X3: 1.0
X4: 1.0
X7: 1.0


In [ ]:
## Resultados varaible Y
for i in range(N_clientes):
  for j in range(N_bodegas):
    if Y[i, j].x > 0:
      print('Y[{}, {}]: {}'.format(i+1, j+1, Y[i, j].x))


Y[1, 4]: 1.0
Y[2, 2]: 1.0
Y[3, 4]: 1.0
Y[4, 7]: 1.0
Y[5, 4]: 1.0
Y[6, 2]: 1.0
Y[7, 7]: 1.0
Y[8, 3]: 1.0
Y[9, 2]: 1.0
Y[10, 2]: 1.0
Y[11, 2]: 1.0
Y[12, 2]: 1.0
Y[13, 2]: 1.0
Y[14, 7]: 1.0
Y[15, 3]: 1.0
Y[16, 4]: 1.0
Y[17, 3]: 1.0
Y[18, 4]: 1.0
Y[19, 2]: 1.0
Y[20, 4]: 1.0
